# Chapter 6: Data loading, storage, and file formats

In [41]:
# Setup
import pandas as pd
import numpy as np

import requests
import os
import sys
import json
import lxml

## 6.1: Reading and writing data in text format

Similar to the tidyverse ecosystem, pandas has several functions for reading data from different formats (e.g., csv) into DataFrames.  These functions can handle type inference (determining if data should be interpreted as strings, numeric, etc), parsing dates and times, and skipping over headers and footers.

To use the examples from the textbook, we need to download the files from the [GitHub repository](https://github.com/wesm/pydata-book).

In [8]:
# Download the first example file
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex1.csv'
request = requests.get(url)

In [10]:
if request.status_code == 200:
    with open('ch06_ex01.csv', 'w') as f:
        f.write(request.text)

In [11]:
# Read into DataFrame
ex1_df = pd.read_csv('ch06_ex01.csv')
ex1_df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [12]:
# We can specify column names ourselves, very similar to tidyverse
names = ['A', 'B', 'C', 'D', 'message']
pd.read_csv('ch06_ex01.csv', names=names)

,A,B,C,D,message
0,a,b,c,d,message
1,1,2,3,4,hello
2,5,6,7,8,world
3,9,10,11,12,foo


In [13]:
# If you have a file with missing data, we can specify which values should be recoded as NA
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex5.csv'
request = requests.get(url)

In [14]:
if request.status_code == 200:
    with open('ch06_ex05.csv', 'w') as f:
        f.write(request.text)

In [18]:
df = pd.read_csv('ch06_ex05.csv', na_values=['NA', 'foo'])
df

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [21]:
# If we want to read chunks of data, we can read just a few lines
pd.read_csv('ch06_ex05.csv', nrows=1)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN


### Writing data out to text format

In [23]:
# Data can be written as csv output
df.to_csv('ch06_ex05_output.csv')

In [24]:
# Read back in the same file
pd.read_csv('ch06_ex05_output.csv')

,Unnamed: 0,something,a,b,c,d,message
0,0,one,1,2,3.0,4,NaN
1,1,two,5,6,NaN,8,world
2,2,three,9,10,11.0,12,NaN


In [25]:
# We can specify how we want missing values to be handled
df.to_csv('ch06_ex05_output.csv', na_rep='NULL')

In [26]:
# Read back in the same file-- specify sentinel for missing data
pd.read_csv('ch06_ex05_output.csv', na_values='NULL')

,Unnamed: 0,something,a,b,c,d,message
0,0,one,1,2,3.0,4,NaN
1,1,two,5,6,NaN,8,world
2,2,three,9,10,11.0,12,NaN


In [29]:
# If we want to experiment with different methods, use sys.stdout to print to screen
df.to_csv(sys.stdout, na_rep='NULL', index=False)

something,a,b,c,d,message
one,1,2,3.0,4,NULL
two,5,6,NULL,8,world
three,9,10,11.0,12,NULL


### JSON data

In [37]:
# Create example JSON data-- parser expects double quotes
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
{"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

In [38]:
# Load JSON data
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 25, 'pet': 'Zuko'},
  {'name': 'Katie', 'age': 33, 'pet': 'Cisco'}]}

In [39]:
# Create an example DataFrame of siblings
pd.DataFrame(result['siblings'], columns=['name', 'age', 'pet'])

,name,age,pet
0,Scott,25,Zuko
1,Katie,33,Cisco


### XML and HTML: web scraping

The textbook example for scraping XML data from New York Metro website is quite outdated, so I'll skip this section for now and add to it later when needed.

## 6.2: Binary data formats

Python has a built in method called `pickel` serialization to store complex data structures in linear format that can be stored locally or transmitted over a network (similar to R's RDS).  This will allow us to write data in `pickel` format, then parse the data back as a DataFrame `pandas.load()`.

Another option for storing large amounts of complex data is HDF5 (hierarchical data format 5).  HDF5 files contain an internal node-like format that connects data and associated metadata, similar to how files might be stored on your computer.

## 6.3 Interacting with HTML and web APIs

The second addition of the textbook uses the GitHub API in an example, which I'll also use here.

In [47]:
# Grab the last 30 pandas issues on GitHub
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)

if resp.status_code == 200:
    print('Download successful!')
else:
    print('Error: Check connection or URL.')

Download successful!


In [50]:
# Parse JSON data
data = resp.json()

# Get the title of the first issue
data[0]['title']

'BUG attempt to fix GH46575'

In [52]:
# Each entry is a dictionary
data[0]

{'url': 'https://api.github.com/repos/pandas-dev/pandas/issues/46644',
 'repository_url': 'https://api.github.com/repos/pandas-dev/pandas',
 'labels_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/46644/labels{/name}',
 'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/46644/comments',
 'events_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/46644/events',
 'html_url': 'https://github.com/pandas-dev/pandas/pull/46644',
 'id': 1193371305,
 'node_id': 'PR_kwDOAA0YD841qzcs',
 'number': 46644,
 'title': 'BUG attempt to fix GH46575',
 'user': {'login': 'arnaudlegout',
  'id': 7670869,
  'node_id': 'MDQ6VXNlcjc2NzA4Njk=',
  'avatar_url': 'https://avatars.githubusercontent.com/u/7670869?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/arnaudlegout',
  'html_url': 'https://github.com/arnaudlegout',
  'followers_url': 'https://api.github.com/users/arnaudlegout/followers',
  'following_url': 'https://api.github.com/users/arnaudlegou

In [53]:
# Grab the number, title, labes, state, crated_at, updated_at, and closed_at fields
github_issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state', 'created_at', 'updated_at', 'closed_at'])
github_issues

,number,title,labels,state,created_at,updated_at,closed_at
0,46644,BUG attempt to fix GH46575,[],open,2022-04-05T15:45:25Z,2022-04-05T15:50:28Z,None
1,46643,Meta issue: CI refactoring,"[{'id': 127681, 'node_id': 'MDU6TGFiZWwxMjc2OD...",open,2022-04-05T07:10:28Z,2022-04-05T08:36:33Z,None
2,46642,DOC: .bfill() (#46631),[],open,2022-04-05T04:23:38Z,2022-04-05T15:49:04Z,None
3,46641,added test for replace with value none,[],open,2022-04-05T03:25:52Z,2022-04-05T03:25:55Z,None
4,46640,homogeneous Period targets bug fix,[],open,2022-04-04T20:38:46Z,2022-04-04T20:38:48Z,None
5,46638,TYP: tighten return type in function any,[],open,2022-04-04T19:46:16Z,2022-04-05T12:27:27Z,None
6,46636,REGR: Replace changes the dtype of other columns,"[{'id': 32815646, 'node_id': 'MDU6TGFiZWwzMjgx...",open,2022-04-04T19:21:07Z,2022-04-05T17:18:00Z,None
7,46635,ENH:,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open,2022-04-04T18:36:22Z,2022-04-05T08:41:48Z,None
8,46634,BUG: Replace changes the dtype of other columns,"[{'id': 32815646, 'node_id': 'MDU6TGFiZWwzMjgx...",open,2022-04-04T16:11:41Z,2022-04-05T09:24:53Z,None
9,46632,BUG: Getting different column types for DataFr...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open,2022-04-04T10:37:46Z,2022-04-04T10:37:46Z,None
